### End to End demo of starting spot instance in AWS

In [53]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [54]:
from aws_setup import *

#### Define parameters

In [3]:
name='fast-ai'

#### Create new VPC by tag name

In [4]:
vpc = create_vpc(name)

#### Create SSH key

In [ ]:
create_ec2_keypair(name)

#### Request Spot instance

In [ ]:
instance_name = f{'{name}-instance'}

In [6]:
instance = create_spot_instance(instance_name, vpc, instance_type='t2.micro'); instance

Waiting on spot fullfillment...
Fullfillment completed. InstanceId: i-09602f7331bed103c
Rebooting...


Request on demand instance (if spot error)

In [68]:
# instance = create_instance(instance_name, vpc, instance_type='t2.micro'); instance

In [74]:
instance = get_instance(f'{instance_name}'); instance

ec2.Instance(id='i-09602f7331bed103c')

#### Attach EBS volume (Optional)

In [75]:
volume_tag = f'{name}-ebs-volume'

In [ ]:
volume = create_volume(volume_tag, size=100)

In [ ]:
_ = attach_volume(instance, volume_tag, device='/dev/xvdf')

#### Create EFS

In [ ]:
efs_tag = f'{name}-efs'

efs = create_efs(efs_tag, vpc)

### SSH into instance

In [77]:
client = connect_to_instance(instance)

Connecting to SSH...
Connected!


#### Mount EBS

In [63]:
upload_path = Path.cwd()/'upload_scripts/mount_ebs.sh'
upload_file(client, str(upload_path), 'mount_ebs.sh')

In [64]:
out, _ = run_command(client, 'chmod 755 mount_ebs.sh')

run_command returned: 



In [81]:
out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf') # no reformatting
# out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf --reformat true')

run_command returned: 
mount: /dev/xvdf is already mounted or /home/ubuntu/ebs_mount_point busy
       /dev/xvdf is already mounted on /home/ubuntu/ebs_mount_point



#### Mount EFS

In [55]:
efs_addr = get_efs_address('fast-ai-efs'); efs_addr

In [60]:
out, _ = run_command(client, 'mkdir ~/efs_mount_point')

run_command returned: 



In [79]:
efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount_point'

In [80]:
out, _ = run_command(client, efs_mount_cmd)

run_command returned: 

